In [32]:
import pandas as pd
import numpy as np
import nltk
import spacy
nltk.download('stopwords')
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

tqdm.pandas(desc="my bar!")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
try:
    nlp = spacy.load('pt_core_news_sm')
except:
    import os
    os.system('python -m spacy download pt_core_news_sm')
    nlp = spacy.load('pt_core_news_sm')

In [34]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['AN_BASE', 'NM_SUBTIPO_PRODUCAO', 'NM_PROJETO', 'DS_RESUMO', 'DS_PALAVRA_CHAVE', 'DS_ABSTRACT', 'DS_KEYWORD', 'NM_AREA_AVALIACAO']].copy()

In [35]:
df_processed.head()

,AN_BASE,NM_SUBTIPO_PRODUCAO,NM_PROJETO,DS_RESUMO,DS_PALAVRA_CHAVE,DS_ABSTRACT,DS_KEYWORD,NM_AREA_AVALIACAO
0,2013,DISSERTAÇÃO,NaN,A POSSIBILIDADE DA OCORRÊNCIA DE SISMOS NO BRA...,"ANÁLISE SÍSMICA, CONCRETO ARMADO, DETALHAMENTO...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
1,2013,DISSERTAÇÃO,NaN,AS OBRAS DE PONTES ESTAIADAS ENVOLVEM UMA TECN...,PONTE ESTAIADA. PONTE DO SABER. ANÁLISE ESTRUT...,NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I
2,2013,TESE,HIPERPLASIA DE CÉLULAS NEUROENDRÓCRINAS EM LAC...,A HIPERPLASIA DE CÉLULAS NEUROENDÓCRINAS DO LA...,1. DOENÇAS PULMONARES INTERSTICIAIS  RADIOGRA...,NEUROENDOCRINE CELL HYPERPLASIA OF INFANCY (NE...,NÃO INFORMADO,MEDICINA II
3,2013,TESE,AVALIAÇÃO ATRAVÉS DAS TÉCNICAS AVANÇADAS DE RM...,ESTUDOS RECENTES TEM DEMONSTRADO ALTERAÇÕES DA...,1. NEURALGIA DO TRIGÊMEO - FISIOPATOLOGIA. 2. ...,RECENT STUDIES HAVE DETECTED LOCAL CHANGES OF ...,NÃO INFORMADO,MEDICINA II
4,2013,DISSERTAÇÃO,NaN,DENTRE AS TÉCNICAS MAIS UTILIZADAS NA DETERMIN...,"PLATAFORMAS FIXAS, ESTRUTURAS OFFSHORE, ESTRUT...",NÃO INFORMADO,NÃO INFORMADO,ENGENHARIAS I


In [36]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 1:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [37]:
df_processed.isna().sum()

ano                     0
subtipo_producao        0
projeto             22280
resumo                  0
palavra_chave           0
abstract                7
keyword                 0
area_avaliacao          0
dtype: int64

In [38]:
df_processed.dropna(subset=['abstract'], inplace=True)

df_processed.fillna({ 'projeto': 'não informado' }, inplace=True)

# df_processed.drop(index=df_processed[
#     df_processed.resumo.str.contains("não informado") | df_processed.palavra_chave.str.contains("não informado") | 
#     df_processed.abstract.str.contains("não informado") | df_processed.keyword.str.contains("não informado") | 
#     df_processed.grande_area_conhecimento.str.contains("não informado")
# ].index, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)

In [39]:
idx = df_processed[(df_processed.palavra_chave == df_processed.keyword) | (df_processed.resumo == df_processed.abstract)].index
df_processed.drop(index=idx, inplace=True)

In [40]:
for item in ['resumo', 'abstract']:
    df_processed[f'{item}'].replace(to_replace ='[0-9]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)

In [41]:
for item in ['palavra_chave', 'keyword']:
    df_processed[f'{item}'].replace(to_replace ='([0-9][. ])', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([;])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='([.])+', value = ',', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='(  )+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(', ')

In [42]:
df_processed.head()

,ano,subtipo_producao,projeto,resumo,palavra_chave,abstract,keyword,area_avaliacao
0,2013,dissertação,não informado,a possibilidade da ocorrência de sismos no bra...,análise sísmica concreto armado detalhamento s...,não informado,não informado,engenharias i
1,2013,dissertação,não informado,as obras de pontes estaiadas envolvem uma tecn...,ponte estaiada ponte do saber análise estrutur...,não informado,não informado,engenharias i
2,2013,tese,hiperplasia de células neuroendrócrinas em lac...,a hiperplasia de células neuroendócrinas do la...,doenças pulmonares intersticiais radiografia d...,neuroendocrine cell hyperplasia of infancy neh...,não informado,medicina ii
3,2013,tese,avaliação através das técnicas avançadas de rm...,estudos recentes tem demonstrado alterações da...,neuralgia do trigêmeo fisiopatologia neuralgia...,recent studies have detected local changes of ...,não informado,medicina ii
4,2013,dissertação,não informado,dentre as técnicas mais utilizadas na determin...,plataformas fixas estruturas offshore estrutur...,não informado,não informado,engenharias i


## Testes

In [43]:
test = df_processed.resumo.copy().head(1000)

In [45]:
def remove_stopwords(text: str) -> str:
    text = wordpunct_tokenize(text)
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in text if item not in stop_words]
    return " ".join(new_text)

In [46]:
test = test.progress_apply(remove_stopwords)

my bar!: 100%|██████████| 1000/1000 [00:00<00:00, 6330.79it/s]


In [47]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(w) for w in wordpunct_tokenize(text)]
    return " ".join(words)

In [48]:
def spc_lem(text: str) -> str:
    words = [token.lemma_ for token in nlp(text) if token.pos_ != 'PUNCT']
    return " ".join(words)

In [49]:
test.progress_apply(lc_stem)

my bar!: 100%|██████████| 1000/1000 [00:01<00:00, 984.85it/s]


0       possibilidad ocorrênc sismo brasil possam comp...
1       obra pont estaiada envolvem tecnolog diferenci...
2       hiperplas células neuroendócrina lact hcnel do...
3       estudo rec demonstrado alteraçõ substância bra...
4       dent técnicas utilizada determinação capacidad...
                              ...                        
1461    campo preservação audiov pass tempo pres perío...
1462    modelo saúd buc atualidad mantém focado atençã...
1463    sistem saúd vem transformando pressionado muda...
1464    alt demand pessoa situaçõ urg procuram serviço...
1467    pres trabalho objetivo compreend naturez front...
Name: resumo, Length: 1000, dtype: object

In [50]:
test.progress_apply(spc_lem)

my bar!: 100%|██████████| 1000/1000 [00:20<00:00, 47.82it/s]


0       possibilidade ocorrência sismo Brasil poder co...
1       obra ponte estaiado envolver tecnologia difere...
2       hiperplasio célula neuroendócrino lactente hcn...
3       estudo recente demonstrar alteração substância...
4       dentre técnica utilizar determinação capacidad...
                              ...                        
1461    Campo preservação audiovisual passar tempo pre...
1462    modelo saúde bucal Atualidade manter focar ate...
1463    sistema saúde vir transformar pressionar mudan...
1464    alto demandar pessoa situação urgente procurar...
1467    presente trabalho objetivo compreender naturez...
Name: resumo, Length: 1000, dtype: object